In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from analysis.datasets import load_entsoe
from analysis.splits import to_train_validation_test_data
from analysis.transformations import scale_power_data
from tabpfn import TabPFNRegressor
from analysis.transformations import add_interval_index, add_lagged_features
from torchinfo import summary
from analysis.TabPFN_copy import evaluate
import torch
import seaborn as sns
from analysis.TabPFN_copy import fit_tail_distribution, plot_cdf_pdf_dynamic, plot_pdf_from_logits
%load_ext autoreload
%autoreload 2

In [2]:
def compute_crps_pytorch(logits, bin_edges, y_values):
    """
    Computes the CRPS for multiple rows of logits and corresponding y-values using PyTorch.

    Args:
        logits: Tensor of shape (N, 5000) - unnormalized logits for each row.
        bin_edges: Tensor of shape (5001,) - common bin edges for all rows.
        y_values: Tensor of shape (N,) - target values for each row.

    Returns:
        Tensor of shape (N,) containing the CRPS values for each row.
    """

    # Convert logits to probabilities using softmax
    probs = torch.softmax(logits, dim=1)  # (N, 5000)

    # Compute CDF (cumulative sum of probabilities)
    cdf = torch.cumsum(probs, dim=1)  # (N, 5000)

    # Compute the indicator function (1 if bin edge >= y, else 0)
    # We need to compare each y_value with bin_edges and broadcast correctly
    indicators = (bin_edges[1:].unsqueeze(0) >= y_values.unsqueeze(1)).float()  # (N, 5000)

    # Step 4: Compute bin widths
    bin_widths = (bin_edges[1:] - bin_edges[:-1]).unsqueeze(0)  # (1, 5000)

    # Step 5: Compute CRPS integral for each row
    crps = torch.sum((cdf - indicators) ** 2 * bin_widths, dim=1)  # (N,)

    return crps

In [3]:
feature_columns = ['ws_10m_loc_mean', 'ws_100m_loc_mean', 'power_t-96']
target_column='power'

entsoe = load_entsoe()
entsoe = add_lagged_features(entsoe)
entsoe = add_interval_index(entsoe)
entsoe.dropna(inplace=True)

Data loaded and transformed successfully. Shape of DataFrame: (78912, 22)


In [5]:
entsoe.head(3)

,power,ws_10m_loc_1,ws_10m_loc_10,ws_10m_loc_2,ws_10m_loc_3,ws_10m_loc_4,ws_10m_loc_5,ws_10m_loc_6,ws_10m_loc_7,ws_10m_loc_8,...,ws_100m_loc_3,ws_100m_loc_4,ws_100m_loc_5,ws_100m_loc_6,ws_100m_loc_7,ws_100m_loc_8,ws_100m_loc_9,ws_100m_loc_mean,power_t-96,interval_index
time,,,,,,,,,,,,,,,,,,,,,
2016-01-02 00:00:00,595.0,2.7200,1.68,2.6400,3.1400,3.110,3.210,3.11,2.63,2.6500,...,5.39,5.200,5.8700,5.650,4.87,4.52,5.32,5.19,1428.0,1
2016-01-02 00:15:00,650.0,2.7975,1.74,2.6625,3.1325,3.185,3.235,3.11,2.65,2.7025,...,5.40,5.315,5.8825,5.645,4.81,4.64,5.33,5.16,1379.0,2
2016-01-02 00:30:00,696.0,2.8750,1.80,2.6850,3.1250,3.260,3.260,3.11,2.67,2.7550,...,5.41,5.430,5.8950,5.640,4.75,4.76,5.34,5.13,1399.0,3


In [6]:
train_end = "2022-12-31 23:45:00"
validation_end = "2023-12-31 23:45:00"

train, validation, test = to_train_validation_test_data(entsoe, train_end, validation_end)
X_train, y_train = train[feature_columns], train[target_column]
X_validation, y_validation = validation[feature_columns], validation[target_column]

# of training observations: 245376 | 77.76%
# of validation observations: 35040 | 11.10%
# of test observations: 35133 | 11.13%


In [12]:
entsoe.max()

power               16676.00
ws_10m_loc_1           13.85
ws_10m_loc_10          14.03
ws_10m_loc_2           14.85
ws_10m_loc_3           14.72
ws_10m_loc_4           15.00
ws_10m_loc_5           14.06
ws_10m_loc_6           14.13
ws_10m_loc_7           12.39
ws_10m_loc_8           12.97
ws_10m_loc_9           12.04
ws_10m_loc_mean        15.84
ws_100m_loc_1          21.09
ws_100m_loc_10         21.60
ws_100m_loc_2          23.15
ws_100m_loc_3          22.62
ws_100m_loc_4          23.33
ws_100m_loc_5          22.07
ws_100m_loc_6          21.95
ws_100m_loc_7          23.85
ws_100m_loc_8          20.39
ws_100m_loc_9          19.61
ws_100m_loc_mean       24.66
power_t-96          16676.00
interval_index         96.00
dtype: float64

In [23]:
# Train 4 models for each quarter
# Define the train-validation splits
splits = {
    "first_q": {
        "train": ("2022-01-01", "2022-03-31"),
        "validation": ("2023-01-01", "2023-03-31"),
        #"train": ("2022-01-01", "2022-01-03"),
        #"validation": ("2023-01-01", "2023-01-03"),
    },
    "second_q": {
        "train": ("2022-04-01", "2022-06-30"),
        "validation": ("2023-04-01", "2023-06-30"),
        #"train": ("2022-01-04", "2022-01-06"),
        #"validation": ("2023-01-04", "2023-01-06"),
    },
    "third_q": {
        "train": ("2022-07-01", "2022-09-30"),
        "validation": ("2023-07-01", "2023-09-30"),
    },
    "fourth_q": {
        "train": ("2022-10-01", "2022-12-31"),
        "validation": ("2023-10-01", "2023-12-31"),
        #"validation": ("2023-12-26", "2023-12-31")
    }
}
# Initialize dictionary to store results
results_dict = {}
summary_stats_overall = {}

# Loop through each quarter
for quarter, dates in splits.items():
    print(f"Processing {quarter}...")

    # Extract train and validation data
    X_train_q = X_train.loc[dates["train"][0]:dates["train"][1]]
    y_train_q = y_train.loc[dates["train"][0]:dates["train"][1]]
    
    X_validation_q = X_validation.loc[dates["validation"][0]:dates["validation"][1]]
    y_validation_q = y_validation.loc[dates["validation"][0]:dates["validation"][1]]

    # Train model
    model = TabPFNRegressor(device='auto', fit_mode='low_memory', random_state=42)
    model.fit(X_train_q, y_train_q)

    # Define quantiles
    quantiles_custom = np.arange(0.1, 1, 0.1)
    probabilities_q = quantiles_custom

    # Predict
    probs_val_q = model.predict(X_validation_q, output_type="full", quantiles=quantiles_custom)
    logits_q = probs_val_q["logits"]
    borders_q = probs_val_q["criterion"].borders
    all_quantiles_q = np.array(probs_val_q["quantiles"])

    # Convert y_validation to tensor
    y_validation_q_torch = torch.tensor(y_validation_q.values, dtype=torch.float32)

    # Compute CRPS and NLL using the 5000 logits and 5001 borders from TabPFN
    crps_values_torch_q = compute_crps_pytorch(logits_q, borders_q, y_validation_q_torch)
    print(f"CRPS shape for {quarter}:", crps_values_torch_q.shape)


    # Initialize max_nll_so_far to a very small number
    max_nll_so_far = float('-9999')
    nll_torch_q = probs_val_q["criterion"].forward(logits_q, y_validation_q_torch)

    # Find the max ignoring inf and NaN
    finite_nlls = nll_torch_q[torch.isfinite(nll_torch_q)]

    if finite_nlls.numel() > 0:  # numel() = number of elements in finite_nlls
        max_nll_so_far = max(max_nll_so_far, finite_nlls.max().item())
        print(f"Updated max_nll_so_far to: {max_nll_so_far} based on the current finite NLLs")

    # Find the indices of the infinite values
    infinite_indices = torch.nonzero(~torch.isfinite(nll_torch_q)).squeeze()

    # Replace infinite values with max_nll_so_far
    infinite_values = nll_torch_q[infinite_indices]
    nll_torch_q[~torch.isfinite(nll_torch_q)] = max_nll_so_far

    # Print out the indices and the values that were replaced
    print(f"Replaced infinite values at indices {infinite_indices.tolist()} in nll_torch_q with the value {max_nll_so_far}.")


    # Fit tails
    yt = entsoe["power"]
    quantile_10 = np.percentile(yt, probabilities_q * 100)
    mu_left_asym, sigma_left_asym = fit_tail_distribution(quantile_10[:2], probabilities_q[:2])
    mu_right_asym, sigma_right_asym = fit_tail_distribution(quantile_10[-2:], probabilities_q[-2:])

    # Initialize lists for CRPS and NLL calculations of the 9 deciles
    crps_cdf_linear_a_full_q = []
    crps_hybrid_cdf_a_full_q = []
    crps_normal_a_full_q = []
    nll_pdf_linear_a_full_q = []
    nll_pdf_hybrid_a_full_q = []
    nll_normal_a_full_q = []
    y_min, y_max = -10000., 20000.

    if np.min(all_quantiles_q) < y_min:
        y_min = np.min(all_quantiles_q)
        print(y_min)
    
    if np.max(all_quantiles_q) > y_max:
        y_max = np.max(all_quantiles_q)
        print(y_max)


    # Iterate through validation samples
    for i in range(y_validation_q.shape[0]):
        quantile_i = all_quantiles_q[:, i]
        y_i = y_validation_q.iloc[i]

        # Compute evaluation metrics
        #cdf_linear, hybrid_cdf, crps_normal, pdf_linear, pdf_hybrid, nll_normal = evaluate(
        #    quantile_i, probabilities_q, y_i, -20, 5, mu_left_asym, sigma_left_asym, mu_right_asym, sigma_right_asym
        #)
        cdf_linear, hybrid_cdf, crps_normal, pdf_linear, pdf_hybrid, nll_normal = evaluate(
            quantile_i, probabilities_q, y_i, y_min, y_max, mu_left_asym, sigma_left_asym, mu_right_asym, sigma_right_asym
        )

        # Store results
        crps_cdf_linear_a_full_q.append(cdf_linear)
        crps_hybrid_cdf_a_full_q.append(hybrid_cdf)
        crps_normal_a_full_q.append(crps_normal)
        nll_pdf_linear_a_full_q.append(pdf_linear)
        nll_pdf_hybrid_a_full_q.append(pdf_hybrid)
        nll_normal_a_full_q.append(nll_normal)

    # Compute mean values
    mean_crps_cdf_linear_q = np.mean(crps_cdf_linear_a_full_q)
    mean_crps_hybrid_cdf_q = np.mean(crps_hybrid_cdf_a_full_q)
    mean_crps_normal_q = np.mean(crps_normal_a_full_q)
    mean_nll_pdf_linear_q = np.mean(nll_pdf_linear_a_full_q)
    mean_nll_pdf_hybrid_q = np.mean(nll_pdf_hybrid_a_full_q)
    mean_nll_normal_q = np.mean(nll_normal_a_full_q)

    # Print mean values
    print(f"Mean CRPS for {quarter} - CDF Linear: {mean_crps_cdf_linear_q:.4f}")
    print(f"Mean CRPS for {quarter} - Hybrid CDF: {mean_crps_hybrid_cdf_q:.4f}")
    print(f"Mean CRPS for {quarter} - Normal: {mean_crps_normal_q:.4f}")
    print(f"Mean NLL for {quarter} - PDF Linear: {mean_nll_pdf_linear_q:.4f}")
    print(f"Mean NLL for {quarter} - PDF Hybrid: {mean_nll_pdf_hybrid_q:.4f}")
    print(f"Mean NLL for {quarter} - Normal: {mean_nll_normal_q:.4f}")

    # Store results in dictionary
    result_q = {
        'CRPS Linear': crps_cdf_linear_a_full_q,
        'CRPS Hybrid': crps_hybrid_cdf_a_full_q,
        'CRPS Normal': crps_normal_a_full_q,
        'CRPS (5000 quantiles)': crps_values_torch_q,
        'NLL Linear': nll_pdf_linear_a_full_q,
        'NLL Hybrid': nll_pdf_hybrid_a_full_q,
        'NLL Normal': nll_normal_a_full_q,
        'NLL (5000 quantiles)': nll_torch_q,
        'y values': y_validation_q_torch,
    }
        
        
        
    # Convert to DataFrame and round values
    results_q = pd.DataFrame(result_q).round(8)
    results_dict[quarter] = results_q

    summary_stats_q = {
        'CRPS Linear': [mean_crps_cdf_linear_q, np.min(crps_cdf_linear_a_full_q), np.max(crps_cdf_linear_a_full_q), np.median(crps_cdf_linear_a_full_q)],
        'CRPS Hybrid': [mean_crps_hybrid_cdf_q, np.min(crps_hybrid_cdf_a_full_q), np.max(crps_hybrid_cdf_a_full_q), np.median(crps_hybrid_cdf_a_full_q)],
        'CRPS Normal': [mean_crps_normal_q, np.min(crps_normal_a_full_q), np.max(crps_normal_a_full_q), np.median(crps_normal_a_full_q)],
        'CRPS (5000 quantiles)': [crps_values_torch_q.mean().item(), np.min(crps_values_torch_q.cpu().numpy()), np.max(crps_values_torch_q.cpu().numpy()), np.median(crps_values_torch_q.cpu().numpy())],
        'NLL Linear': [mean_nll_pdf_linear_q, np.min(nll_pdf_linear_a_full_q), np.max(nll_pdf_linear_a_full_q), np.median(nll_pdf_linear_a_full_q)],
        'NLL Hybrid': [mean_nll_pdf_hybrid_q, np.min(nll_pdf_hybrid_a_full_q), np.max(nll_pdf_hybrid_a_full_q), np.median(nll_pdf_hybrid_a_full_q)],
        'NLL Normal': [mean_nll_normal_q, np.min(nll_normal_a_full_q), np.max(nll_normal_a_full_q), np.median(nll_normal_a_full_q)],
        'NLL (5000 quantiles)': [nll_torch_q.mean().item(), np.min(nll_torch_q.cpu().numpy()), np.max(nll_torch_q.cpu().numpy()), np.median(nll_torch_q.cpu().numpy())],
    }

    # Convert the summary stats into a DataFrame
    summary_stats_q_pd = pd.DataFrame(summary_stats_q, index=['Mean', 'Min', 'Max', 'Median'])
    summary_stats_overall[quarter] = summary_stats_q_pd

    # Print summary
    print("✅ Processing complete. Results stored in `results_dict`.")

Processing first_q...


c:\Users\Minu\Documents\master-thesis\.venv\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
c:\Users\Minu\Documents\master-thesis\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Minu\Documents\master-thesis\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Minu\Documents\master-thesis\.venv\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This func

CRPS shape for first_q: torch.Size([8640])
Updated max_nll_so_far to: 20.700092315673828 based on the current finite NLLs
Replaced infinite values at indices [] in nll_torch_q with the value 20.700092315673828.


C:\Users\Minu\Documents\master-thesis\src\analysis\TabPFN_copy.py:359: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  crps_value, _ = quad(integrand, y_min, y_max)
C:\Users\Minu\Documents\master-thesis\src\analysis\TabPFN_copy.py:359: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  crps_value, _ = quad(integrand, y_min, y_max)


Mean CRPS for first_q - CDF Linear: 878.3663
Mean CRPS for first_q - Hybrid CDF: 3747.7258
Mean CRPS for first_q - Normal: 836.3055
Mean NLL for first_q - PDF Linear: 10.0371
Mean NLL for first_q - PDF Hybrid: 11.9538
Mean NLL for first_q - Normal: 16.3745
✅ Processing complete. Results stored in `results_dict`.
Processing second_q...


...

Mean CRPS for third_q - CDF Linear: 743.8101
Mean CRPS for third_q - Hybrid CDF: 1517.9408
Mean CRPS for third_q - Normal: 691.4791
Mean NLL for third_q - PDF Linear: 9.5101
Mean NLL for third_q - PDF Hybrid: 8.5721
Mean NLL for third_q - Normal: 11.7756
✅ Processing complete. Results stored in `results_dict`.
Processing fourth_q...


c:\Users\Minu\Documents\master-thesis\.venv\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
c:\Users\Minu\Documents\master-thesis\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Minu\Documents\master-thesis\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Minu\Documents\master-thesis\.venv\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This func

CRPS shape for fourth_q: torch.Size([8832])
Updated max_nll_so_far to: 21.09278106689453 based on the current finite NLLs
Replaced infinite values at indices [] in nll_torch_q with the value 21.09278106689453.


C:\Users\Minu\Documents\master-thesis\src\analysis\TabPFN_copy.py:359: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  crps_value, _ = quad(integrand, y_min, y_max)
C:\Users\Minu\Documents\master-thesis\src\analysis\TabPFN_copy.py:359: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  crps_value, _ = quad(integrand, y_min, y_max)


Mean CRPS for fourth_q - CDF Linear: 1144.1682
Mean CRPS for fourth_q - Hybrid CDF: 3596.0906
Mean CRPS for fourth_q - Normal: 1134.9627
Mean NLL for fourth_q - PDF Linear: 10.2689
Mean NLL for fourth_q - PDF Hybrid: 9.9651
Mean NLL for fourth_q - Normal: 22.1410
✅ Processing complete. Results stored in `results_dict`.


In [15]:
entsoe.power.max()

16676.0

In [24]:
epsilon = 1e-3
lag=96
transformation = "None"

# Table for metadata

split_rows = []
for quarter, dates in splits.items():
    # Create a row with both 'train' and 'validation' in the same row
    split_rows.append({
        'quarter': quarter,
        'train_start_date': dates['train'][0],
        'train_end_date': dates['train'][1],
        'validation_start_date': dates['validation'][0],
        'validation_end_date': dates['validation'][1],
        'random_seed': 42,
        'num_splits': 4,
        'epsilon': epsilon,
        'lag': lag,
        'transformation': transformation,
        'features': feature_columns,
        'model_type': "TabPFNRegressor",
        'device': "auto",
        'fit_mode': "low_memory",
        "ignore_pretaining_limits": "False"
    })

meta_info_df = pd.DataFrame(split_rows)

In [25]:
# Combine all DataFrames from results_dict into one big DataFrame
all_quarters_df = pd.concat(results_dict.values(), ignore_index=True)

# Compute mean, min, median, and max for all numeric columns
summary_stats = all_quarters_df.describe().loc[['mean', 'min', '50%', 'max']].rename(index={'50%': 'median'})

# Compute mean of "CRPS (5000 quantiles)" separately
crps_mean = all_quarters_df["CRPS (5000 quantiles)"].mean()

# Compute mean of "NLL (5000 quantiles)" separately
nll_mean = all_quarters_df["NLL (5000 quantiles)"].mean()

# Convert them into separate DataFrames
crps_mean_df = pd.DataFrame({"Metric": ["Mean CRPS (5000 quantiles)"], "Value": [crps_mean]})
nll_mean_df = pd.DataFrame({"Metric": ["Mean NLL (5000 quantiles)"], "Value": [nll_mean]})

In [26]:
# Define the output Excel file name
output_excel_file = '../../../OneDrive/Arbeit/HTWG/Master/results/TabPFN/results_ws10m_ws100m_pt_96_unstandardized_power.xlsx'
#output_excel_file = '../../../OneDrive/Arbeit/HTWG/Master/results/TabPFN/tests/2_results_ws10m_ws100m_pt_96.xlsx'

# Write everything to the Excel file
with pd.ExcelWriter(output_excel_file) as writer:
    # Write each split DataFrame to its own sheet
    for quarter, df in results_dict.items():
        df.to_excel(writer, sheet_name=quarter, index=False)
        summary_stats_overall[quarter].to_excel(writer, sheet_name=quarter, startcol=len(df.columns) + 3, index=True)

    # Write the meta_info DataFrame
    meta_info_df.to_excel(writer, sheet_name='meta_info', index=False)
    
    # Write summary statistics
    summary_stats.to_excel(writer, sheet_name="summary_stats")
    
    # Append the mean CRPS and NLL separately below the summary stats
    crps_mean_df.to_excel(writer, sheet_name="summary_stats", startrow=len(summary_stats) + 2, index=False)
    nll_mean_df.to_excel(writer, sheet_name="summary_stats", startrow=len(summary_stats) + 4, index=False)

print(f'Summary statistics and mean CRPS/NLL added to "summary_stats" sheet in {output_excel_file}')

Summary statistics and mean CRPS/NLL added to "summary_stats" sheet in ../../../OneDrive/Arbeit/HTWG/Master/results/TabPFN/results_ws10m_ws100m_pt_96_unstandardized_power.xlsx


In [27]:
print("done writing the results of ws_10m, ws_100m, P_t-96_unstandardized_power to excel file")

done writing the results of ws_10m, ws_100m, P_t-96_unstandardized_power to excel file


z=(y-mu)/sigma or y= mu + sigma * z, mu = 3760, sigma = 3388, TabPFN lowest and highest bin borders are z_min = - 98, z_max = 86. Hence
y_min = 3760 - 3388*98 = -328'264, y_max = 3760 + 3388*86 = 295'128

In [17]:
entsoe["power"].std()

3388.4129789136728

In [19]:
y_min = 3760 - 3388*98
y_min

-328264

In [20]:
y_max = 3760 + 3388*86
y_max

295128